In [ ]:
# This cell imports the drive library and mounts your Google Drive as a VM local drive. You can access to your Drive files 
# using this path "/content/gdrive/My Drive/"

from google.colab import drive
drive.mount('/content/gdrive') # ,force_remount=True

In [ ]:
# Not Necessary cell
# List the content of your local computer folder 
!ls -1t "/content/gdrive/My Drive/darknet/data"

In [ ]:
#Rajout d'un préfixe
!sed -e 's/.*/client&/' "/content/gdrive/My Drive/darknet/data/yahourt.txt" > "/content/gdrive/My Drive/darknet/data/newFich.txt"

#

## STEP 4-A. Cloning and compiling Darkent. ONLY NEEDS TO BE RUN ON THE FIRST EXECUTION!!
In this step we'll clone the darkent repo and compile it. 
* Clone Repo
* Compile Darknet
* Copy compiled version to Drive

When compiling ends, your output last line has to be something like this:

_`g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv` -DGPU (...)_`

In [ ]:
# Leave this code uncommented on the very first run of your notebook or if you ever need to recompile darknet again.
# Comment this code on the future runs.
!git clone https://github.com/kriyeng/darknet/
%cd darknet

# Check the folder
!ls

# I have a branch where I have done the changes commented above
!git checkout feature/google-colab

#Compile Darknet
!make

#Copies the Darknet compiled version to Google drive
!cp ./darknet /content/gdrive/My\ Drive/darknet/bin/darknet

## STEP 4-B. Copying the compiled version of Darknet from Drive. UNCOMMENT AFTER FIRST EXECUTION
Copy the darknet compiled version from drive to the VM.
* Make the local darknet folder
* Copy the darknet file
* Set execution permissions

In [ ]:
# Uncomment after the first run, when you have a copy of compiled darkent in your Google Drive

# Makes a dir for darknet and move there
!mkdir darknet
%cd darknet

# Copy the Darkent compiled version to the VM local drive
!cp /content/gdrive/My\ Drive/darknet/bin/darknet ./darknet

# Set execution permissions to Darknet
!chmod +x ./darknet


## STEP 5. Runtime configuration finished!
Let's chek it out!

If you are running this notebook for the first time, you can run the following cells in order to check if everything goes as expected!




## Some Utils
Let's add some utils that maybe can be useful.

These utils are:
* imgShow() - Will help us to show an image in the remote VM
* download() - Will allow you to get some file from your notebook in case you need to
* upload() - You can upload files to your current folder on the remote VM.

Thanks to [Ivan Goncharov](https://twitter.com/Ivangrov) for these helpers!

In [ ]:
#download files
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# Not necessary cell
# Get yolov3 weights
!wget https://pjreddie.com/media/files/yolov3.weights

**NOTE:** The following test only will work when the darknet is compiled in the runtime. This demo uses some data from the original darknet folders. For your Object detection projects, you'll have these necessary files on your local folder.

In [ ]:
# Not necessary cell
# Execute darknet using YOLOv3 model with pre-trained weights to detect objects on 'person.jpg'
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont-show

# Show the result using the helper imgShow()
imShow('predictions.jpg')

In [ ]:
# don't go beyond here with Run All
#assert False

## PART 3. Finally, Train



In [ ]:
#!chmod 777 "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny-obj.cfg"
!cp -r "/content/gdrive/My Drive/darknet/data" ./
#!cp -r "/content/gdrive/My Drive/darknet/cfg" ./


In [ ]:
!ls -la "/content/gdrive/My Drive/darknet/data/obj/Tomato" 
#/content/gdrive/My Drive/darknet/data/obj/Tomato/vegetables-1085063_640.jpg

#data/obj/Milk/laitfrigo5.jpg
#data/obj/Milk/irl1.jpg
#data/obj/Tomato/tomatotest1.jpg

In [ ]:
# Not necessary cell
!./darknet detect "/content/gdrive/My Drive/darknet/data/obj.data" "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny-obj.cfg" "/content/gdrive/My Drive/darknet/backup/yolov3-tiny-obj_6000.weights" "/content/gdrive/My Drive/darknet/data/obj/Milk/laitfrigo1.jpg" -dont-show

# Show the result using the helper imgShow()
imShow('predictions.jpg')

In [ ]:
#recuperation du backup virtuel
!cp -r backup "/content/gdrive/My Drive/darknet/backup" 

In [ ]:
# Test du réseau de neuronne
!./darknet detector test "/content/gdrive/My Drive/darknet/data/obj.data" "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny-obj.cfg" "/content/gdrive/My Drive/darknet/backup/yolov3-tiny-obj_6000.weights" -dont_show 
imShow('predictions.jpg')

In [ ]:
# Entrainement du réseau de neuronne
!./darknet detector train "/content/gdrive/My Drive/darknet/data/obj.data" "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny-obj.cfg" "/content/gdrive/My Drive/darknet/yolov3-tiny.conv.15" -dont_show 

## PERFORMANCE TIPS & TRICKS

* **Speed up load times of the runtime:** When everything is checked that works, you can remove cells or comment unnecessary lines of code to make your loading time lower on every run. 

* **How to keep your notebook alive for more time?:** Keep you browser with your notebook open. If you close your browser, your notebook will reach the iddle time and will be removed from Colab cloud Service. (90 minutes)
  

In [ ]:
# Start training at the point where the last runtime finished
!./darknet detector train "/content/gdrive/My Drive/darknet/data/obj.data" "/content/gdrive/My Drive/darknet/cfg/yolov3-tiny-obj.cfg" "/content/gdrive/My Drive/darknet/weights/yolov_last.weights" -dont_show 